In [1]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import optuna
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')  # ignore notifications
import sys
import os


C:\Users\Hp\anaconda3\envs\ankitaenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
df = pd.read_excel('COM.xlsx')
pd.set_option('display.max_columns', None)
df.head()

,listing litle,city,area,location,zone,location_hub,property_type,ownership,size_in_sqft,carpet_area_sqft,private_washroom,public_washroom,floor_no,total_floors,amenities_count,electric_charge_included,water_charge_included,property_age,possession_status,posted_by,lock in period,expected rent increases yearly,negotiable,brokerage,security_deposite,rent_price
0,showroom for rent,nagpur,dharampeth,"dharampeth,nagpur",east,Retail Complex/ Building,showroom,freehold,900 sqft,750 sqft,1,1,1 floor,1 floor,"CCTV,food court(2)",no,yes,5 years,ready to move,housing expert,6 months,0.10,yes,yes,450000.0,75000
1,ready to use office space,nagpur,ramdaspeth,"ramdaspeth, nagpur",east,business park,ready to use office,freehold,600 sqft,500 sqft,1,1,1 floor,1 floor,"lift, cabin,metting room(3)",no,yes,6 years,ready to move,housing expert,2 months,0.05,yes,yes,NaN,15000
2,shop for rent,nagpur,manewada,"manewada, nagpur",south,others,shop,freehold,300 sqft,250 sqft,1,0,ground floor,2 floors,0,no,yes,NaN,ready to move,owner,12 months,0.10,yes,no,25000.0,12000
3,commercial property,nagpur,manewada,"manewada, nagpur",south,others,commercial property,freehold,1350 sqft,1012 sqft,1,1,ground floor,2 floors,0,yes,yes,5 years,ready to move,housing expert,6 months,0.10,yes,yes,40000.0,20000
4,ready to use office space,nagpur,manewada,"vittal nagar ,manewda,nagpur",south,others,ready to use office,freehold,2000 sqft,1800 sqft,1,1,ground floor,2 floors,"water storage, internet,cabin,meeting rooms(4)",yes,yes,10 years,ready to move,owner,6 months,NaN,no,no,200000.0,100000


In [3]:
for col in df.columns:
    print(f"Value counts for column: {col}")
    print(df[col].value_counts())
    print("-" * 40)

Value counts for column: listing litle
listing litle
showroom for rent            243
shop for rent                216
bare shell office space      211
ready to use office space    171
commercial property           84
werehouse                     60
godown for rent               20
Name: count, dtype: int64
----------------------------------------
Value counts for column: city
city
nagpur    1005
Name: count, dtype: int64
----------------------------------------
Value counts for column: area
area
manewada             73
jaitala              71
besa                 67
omkar nagar          59
itwari               56
hingna               52
sitabuldi            52
mahal                47
kharbi               44
mihan                36
pratap nagar         36
ramdaspeth           33
dharampeth           32
gandhibag            31
chatrapati nagar     29
nandanwan            23
sadar                21
dighori              19
somalwada            19
ganeshpeth colony    18
mhalgi nagar     

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   listing litle                   1005 non-null   object 
 1   city                            1005 non-null   object 
 2   area                            1005 non-null   object 
 3   location                        1005 non-null   object 
 4   zone                            1005 non-null   object 
 5   location_hub                    1005 non-null   object 
 6   property_type                   1005 non-null   object 
 7   ownership                       1005 non-null   object 
 8   size_in_sqft                    1005 non-null   object 
 9   carpet_area_sqft                786 non-null    object 
 10  private_washroom                1005 non-null   int64  
 11  public_washroom                 1005 non-null   int64  
 12  floor_no                        10

In [5]:
df = df.drop('location', axis=1)

In [6]:
df = df.replace(" ", 0).fillna(0)

In [7]:
import pandas as pd
import re

def floor_to_int_list(floor_str):
    floor_str = str(floor_str).lower()
    floor_str = floor_str.replace('floor', '').replace('floors', '').replace(' ', '')
    # Replace ground floor / GF with 0
    floor_str = floor_str.replace('ground', '0').replace('gf', '0')
    # Split multiple floors
    parts = re.split(r'[,]', floor_str)
    floor_numbers = []
    for p in parts:
        try:
            floor_numbers.append(int(p))
        except:
            continue
    # Return as comma-separated string
    if floor_numbers:
        return ','.join(map(str, sorted(floor_numbers)))
    return None

# Apply to your DataFrame
df['floor_no'] = df['floor_no'].apply(floor_to_int_list)

# Show result
print(df[['floor_no']])


       floor_no
0             1
1             1
2             0
3             0
4             0
...         ...
1000          2
1001  0,1,2,3,4
1002          1
1003          2
1004          2

[1005 rows x 1 columns]


In [8]:
import pandas as pd

def total_floors_to_int(floor_str):
    try:
        # Remove 'floors' or 'floor' and spaces, then convert to int
        return int(str(floor_str).lower().replace('floors', '').replace('floor', '').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['total_floors'] = df['total_floors'].apply(total_floors_to_int)




In [9]:
import pandas as pd

# Function to convert size to integer
def size_to_int(size_str):
    try:
        # Remove 'sqft' or 'sq.ft' and spaces, then convert to int
        return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['size_in_sqft'] = df['size_in_sqft'].apply(size_to_int)

# Check result
print(df[['size_in_sqft']].head(20))


    size_in_sqft
0            900
1            600
2            300
3           1350
4           2000
5            200
6           6000
7            550
8           3500
9           1300
10          2600
11          1600
12          1000
13           550
14          2100
15           950
16          1000
17          1200
18          1600
19           250


In [10]:
import pandas as pd

# Function to convert size to integer
def size_to_int(size_str):
    try:
        # Remove 'sqft' or 'sq.ft' and spaces, then convert to int
        return int(str(size_str).lower().replace('sqft','').replace('sq.ft','').strip())
    except:
        return None  # in case of unexpected value

# Apply to DataFrame
df['carpet_area_sqft'] = df['carpet_area_sqft'].apply(size_to_int)

# Check result
print(df[['carpet_area_sqft']].head(20))

    carpet_area_sqft
0                750
1                500
2                250
3               1012
4               1800
5                175
6               4000
7                500
8               3200
9               1250
10              2600
11              1500
12               900
13               550
14              1900
15               800
16              1000
17              1152
18              1600
19               150


In [11]:
print(df['amenities_count'].value_counts())

amenities_count
parking, vastu(2)                                                                   34
lift, cabin,metting room(3)                                                         30
DG and UPS , parking, staircases, CCTV, power backup, reception area, pantry (8)    21
water storage, parking, staircase, security(4)                                      21
water storage, parking(2)                                                           21
                                                                                    ..
CCTV,parking(2)                                                                      1
security, vastu(2)                                                                   1
water storage, CCTV, vastu(3)                                                        1
parking, staircase,water storage,CCTV (4)                                            1
staircase, lift, parking, water supply (4)                                           1
Name: count, Length: 94, dt

In [12]:
# Function to extract amenities as a list
def extract_amenities_list(text):
    text = str(text).lower()
    # Remove counts in parentheses
    text = re.sub(r'\(\d+\)', '', text)
    # Split by comma and strip spaces
    amenities = [x.strip() for x in text.split(',') if x.strip() != '']
    return amenities

# Apply function
df['amenities_list'] = df['amenities_count'].apply(extract_amenities_list)

# Get unique amenities across all rows
all_amenities = set([amenity for sublist in df['amenities_list'] for amenity in sublist])

# Create binary columns
for amenity in all_amenities:
    df[amenity.replace(' ', '_')] = df['amenities_list'].apply(lambda x: 1 if amenity in x else 0)

# Drop intermediate column if needed
df = df.drop(columns=['amenities_list'])


In [13]:

import re

def extract_amenities_list(text):
    text = str(text).lower()
    # Remove counts in parentheses
    text = re.sub(r'\(\d+\)', '', text)
    # Split by comma and strip spaces
    amenities = [x.strip() for x in text.split(',') if x.strip() != '']
    return amenities

# Create a list of all amenities
all_amenities = set([amenity for sublist in df['amenities_count'].apply(extract_amenities_list) for amenity in sublist])

# Optionally, replace spaces with underscores for column names
binary_columns = [amenity.replace(' ', '_') for amenity in sorted(all_amenities)]

print(binary_columns)


['0', 'cabin', 'cctv', 'ccvt', 'cebin', 'dg', 'dg_and_ups', 'fire_extinghuishers', 'fire_extinguisher', 'fire_extinguishers', 'fire_safety', 'fire_safty', 'fire_sefty', 'fire_sensors', 'food_court', 'furnishing', 'internate', 'internet', 'lift', 'meeting_room', 'meeting_rooms', 'metting_room', 'oxygen_duct', 'paarking', 'panetry', 'pantry', 'parking', 'power_backup', 'reception_area', 'security', 'sqcurity', 'staircase', 'staircases', 'staricase', 'vastu', 'water_storae', 'water_storage', 'water_supply']


In [14]:
df.head()

,listing litle,city,area,zone,location_hub,property_type,ownership,size_in_sqft,carpet_area_sqft,private_washroom,public_washroom,floor_no,total_floors,amenities_count,electric_charge_included,water_charge_included,property_age,possession_status,posted_by,lock in period,expected rent increases yearly,negotiable,brokerage,security_deposite,rent_price,fire_extinguisher,fire_extinguishers,paarking,panetry,food_court,staircases,cabin,staricase,lift,0,water_storage,meeting_rooms,dg,fire_safety,metting_room,ccvt,security,fire_sefty,cctv,oxygen_duct,staircase,furnishing,fire_safty,vastu,cebin,reception_area,meeting_room,fire_extinghuishers,internet,water_supply,fire_sensors,power_backup,sqcurity,dg_and_ups,parking,water_storae,pantry,internate
0,showroom for rent,nagpur,dharampeth,east,Retail Complex/ Building,showroom,freehold,900,750,1,1,1,1,"CCTV,food court(2)",no,yes,5 years,ready to move,housing expert,6 months,0.10,yes,yes,450000.0,75000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ready to use office space,nagpur,ramdaspeth,east,business park,ready to use office,freehold,600,500,1,1,1,1,"lift, cabin,metting room(3)",no,yes,6 years,ready to move,housing expert,2 months,0.05,yes,yes,0.0,15000,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,shop for rent,nagpur,manewada,south,others,shop,freehold,300,250,1,0,0,2,0,no,yes,0,ready to move,owner,12 months,0.10,yes,no,25000.0,12000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,commercial property,nagpur,manewada,south,others,commercial property,freehold,1350,1012,1,1,0,2,0,yes,yes,5 years,ready to move,housing expert,6 months,0.10,yes,yes,40000.0,20000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ready to use office space,nagpur,manewada,south,others,ready to use office,freehold,2000,1800,1,1,0,2,"water storage, internet,cabin,meeting rooms(4)",yes,yes,10 years,ready to move,owner,6 months,0.00,no,no,200000.0,100000,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [15]:
df.to_csv('new_file.csv', index=False) 

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 63 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   listing litle                   1005 non-null   object 
 1   city                            1005 non-null   object 
 2   area                            1005 non-null   object 
 3   zone                            1005 non-null   object 
 4   location_hub                    1005 non-null   object 
 5   property_type                   1005 non-null   object 
 6   ownership                       1005 non-null   object 
 7   size_in_sqft                    1005 non-null   int64  
 8   carpet_area_sqft                1005 non-null   int64  
 9   private_washroom                1005 non-null   int64  
 10  public_washroom                 1005 non-null   int64  
 11  floor_no                        1005 non-null   object 
 12  total_floors                    10

In [17]:
import pandas as pd

# Mapping of duplicates → clean column name
amenities_mapping = {
    'fire_extinghuishers': 'fire_extinguishers',
    'fire_extinguisher': 'fire_extinguishers',
    'staricase': 'staircase',
    'staircases': 'staircase',
    'staircase': 'staircase',
    'ccvt': 'cctv',
    'sqcurity': 'security',
    'internate': 'internet',
    'internate': 'internet',
    'cebin': 'cabin',
    'cebin': 'cabin',
    'panetry': 'pantry',
    'fire_safty': 'fire_safety',
    'fire_sefty': 'fire_safety',
    'water_storae': 'water_storage',
    'meeting_room': 'meeting_rooms',
    'metting_room': 'meeting_rooms',
    'meeting_rooms': 'meeting_rooms',
    'paarking': 'parking'
}

# Loop through mapping and combine columns
for old_col, new_col in amenities_mapping.items():
    if old_col in df.columns:
        if new_col in df.columns:
            df[new_col] = df[new_col] | df[old_col]  # combine using OR for binary
        else:
            df[new_col] = df[old_col]
        df.drop(columns=[old_col], inplace=True)

# Check remaining columns
print(df.columns)


Index(['listing litle', 'city', 'area', 'zone', 'location_hub',
       'property_type', 'ownership', 'size_in_sqft', 'carpet_area_sqft',
       'private_washroom', 'public_washroom', 'floor_no', 'total_floors',
       'amenities_count', 'electric_charge_included', 'water_charge_included',
       'property_age', 'possession_status', 'posted_by', 'lock in period',
       'expected rent increases yearly', 'negotiable', 'brokerage',
       'security_deposite', 'rent_price', 'fire_extinguishers', 'food_court',
       'cabin', 'lift', '0', 'water_storage', 'dg', 'fire_safety', 'security',
       'cctv', 'oxygen_duct', 'furnishing', 'vastu', 'reception_area',
       'internet', 'water_supply', 'fire_sensors', 'power_backup',
       'dg_and_ups', 'parking', 'pantry'],
      dtype='object')


In [18]:

# Define a mapping for similar items
mapping = {
    'Commercial Project': 'commercial project',
    'Business Park': 'business park',
    'Residential Project': 'residential',
    'Retail Complex/ Building': 'retail complex/building',
    'Market/ High Street': 'market/high street',
    'IT Park': 'it park'
    # "others" remains unchanged
}

# Standardize the column values using the mapping
df['location_hub'] = df['location_hub'].replace(mapping)

# Get value counts
counts = df['location_hub'].value_counts()
print(counts)

location_hub
commercial project         320
others                     295
retail complex/building    110
market/high street         100
business park               79
it park                     59
residential                 31
Residential                 11
Name: count, dtype: int64


In [19]:
df['property_age'] = df['property_age'].str.replace(' years', '', regex=False) \
                                       .str.replace(' year', '', regex=False) \
                                       .fillna(0) \
                                       .astype(int)


In [20]:
print(df['property_age'].value_counts())

property_age
5     275
3     108
10    107
2      89
15     88
0      78
4      63
1      58
6      48
20     36
12     32
30     12
7      11
Name: count, dtype: int64


In [21]:
# Clean 'lock in period' column
df['lock_in_period_in_months'] = df['lock in period'].str.replace('months', '', regex=False) \
                                           .str.replace('month', '', regex=False) \
                                           .str.strip() \
                                           .fillna(0) \
                                           .astype(int)

# Check the cleaned values
print(df['lock_in_period_in_months'].value_counts())
# Drop the original column
df = df.drop(columns=['lock in period'])


lock_in_period_in_months
2     261
6     235
12    110
3      88
1      81
11     60
4      59
0      45
10     29
8      20
36     17
Name: count, dtype: int64


In [22]:
df = df.drop('security_deposite', axis=1)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   listing litle                   1005 non-null   object 
 1   city                            1005 non-null   object 
 2   area                            1005 non-null   object 
 3   zone                            1005 non-null   object 
 4   location_hub                    1005 non-null   object 
 5   property_type                   1005 non-null   object 
 6   ownership                       1005 non-null   object 
 7   size_in_sqft                    1005 non-null   int64  
 8   carpet_area_sqft                1005 non-null   int64  
 9   private_washroom                1005 non-null   int64  
 10  public_washroom                 1005 non-null   int64  
 11  floor_no                        1005 non-null   object 
 12  total_floors                    10

In [24]:
# Select only object type columns
df_object = df.select_dtypes(include='object')

# Check the columns
print(df_object.columns)
print(df_object.head())


Index(['listing litle', 'city', 'area', 'zone', 'location_hub',
       'property_type', 'ownership', 'floor_no', 'amenities_count',
       'electric_charge_included', 'water_charge_included',
       'possession_status', 'posted_by', 'negotiable', 'brokerage'],
      dtype='object')
               listing litle    city        area   zone  \
0          showroom for rent  nagpur  dharampeth   east   
1  ready to use office space  nagpur  ramdaspeth   east   
2              shop for rent  nagpur    manewada  south   
3        commercial property  nagpur    manewada  south   
4  ready to use office space  nagpur    manewada  south   

              location_hub        property_type ownership floor_no  \
0  retail complex/building             showroom  freehold        1   
1            business park  ready to use office  freehold        1   
2                   others                 shop  freehold        0   
3                   others  commercial property  freehold        0   
4           

In [25]:
print("--- Feature Engineering: One-Hot Encoding Categorical Features ---")
categorical_features = ['listing litle', 'city', 'area', 'zone', 'location_hub',
       'property_type', 'ownership', 'floor_no', 'amenities_count',
       'electric_charge_included', 'water_charge_included',
       'possession_status', 'posted_by', 'negotiable', 'brokerage']
for feature in categorical_features:
    if feature in df.columns:
        df = pd.get_dummies(df, columns=[feature], drop_first=True)
        print(f"One-hot encoded '{feature}'.")
    else:
        print(f"Warning: Categorical feature '{feature}' not found in DataFrame for encoding.")


# Transform 'Rent' using natural logarithm to reduce skewness
print("--- Feature Engineering: Log Transformation of 'Rent' ---")
# Add a small constant to Rent to avoid log(0) if any rent value is 0
df['rent_price'] = np.log1p(df['rent_price']) # Using log1p which is log(1+x)
print("Applied log transformation to 'Rent' column.")

print("\n--- DataFrame after Feature Engineering ---")
print(df.head())
print(df.info())

--- Feature Engineering: One-Hot Encoding Categorical Features ---
One-hot encoded 'listing litle'.
One-hot encoded 'city'.
One-hot encoded 'area'.
One-hot encoded 'zone'.
One-hot encoded 'location_hub'.
One-hot encoded 'property_type'.
One-hot encoded 'ownership'.
One-hot encoded 'floor_no'.
One-hot encoded 'amenities_count'.
One-hot encoded 'electric_charge_included'.
One-hot encoded 'water_charge_included'.
One-hot encoded 'possession_status'.
One-hot encoded 'posted_by'.
One-hot encoded 'negotiable'.
One-hot encoded 'brokerage'.
--- Feature Engineering: Log Transformation of 'Rent' ---
Applied log transformation to 'Rent' column.

--- DataFrame after Feature Engineering ---
   size_in_sqft  carpet_area_sqft  private_washroom  public_washroom  \
0           900               750                 1                1   
1           600               500                 1                1   
2           300               250                 1                0   
3          1350          

In [26]:
# Define features (X) and target (y)
X = df.drop('rent_price', axis=1)
y = df['rent_price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nData split into training (80%) and testing (20%) sets.")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Data split into training (80%) and testing (20%) sets.
X_train shape: (804, 202), y_train shape: (804,)
X_test shape: (201, 202), y_test shape: (201,)


In [27]:
# Select all numeric columns (int and float)
df_numeric = df.select_dtypes(include=['int64', 'float64'])

# Check the columns
print(df_numeric.columns)
print(df_numeric.head())


Index(['size_in_sqft', 'carpet_area_sqft', 'private_washroom',
       'public_washroom', 'total_floors', 'property_age',
       'expected rent increases yearly', 'rent_price', 'fire_extinguishers',
       'food_court', 'cabin', 'lift', '0', 'water_storage', 'dg',
       'fire_safety', 'security', 'cctv', 'oxygen_duct', 'furnishing', 'vastu',
       'reception_area', 'internet', 'water_supply', 'fire_sensors',
       'power_backup', 'dg_and_ups', 'parking', 'pantry',
       'lock_in_period_in_months'],
      dtype='object')
   size_in_sqft  carpet_area_sqft  private_washroom  public_washroom  \
0           900               750                 1                1   
1           600               500                 1                1   
2           300               250                 1                0   
3          1350              1012                 1                1   
4          2000              1800                 1                1   

   total_floors  property_age  expecte

In [28]:
numerical_cols = ['size_in_sqft', 'carpet_area_sqft', 'private_washroom',
       'public_washroom', 'total_floors', 'property_age',
       'expected rent increases yearly', 'rent_price', 'fire_extinguishers',
       'food_court', 'cabin', 'lift', '0', 'water_storage', 'dg',
       'fire_safety', 'security', 'cctv', 'oxygen_duct', 'furnishing', 'vastu',
       'reception_area', 'internet', 'water_supply', 'fire_sensors',
       'power_backup', 'dg_and_ups', 'parking', 'pantry',
       'lock_in_period_in_months']
# Filter numerical_cols to only include columns actually present in X_train
numerical_cols_present = [col for col in numerical_cols if col in X_train.columns]

# 1. Replace empty strings with NaN
X_train = X_train.replace(r'^\s*$', np.nan, regex=True)
X_test = X_test.replace(r'^\s*$', np.nan, regex=True)

# 2. Convert all numeric columns to float
X_train[numerical_cols_present] = X_train[numerical_cols_present].apply(pd.to_numeric, errors='coerce')
X_test[numerical_cols_present] = X_test[numerical_cols_present].apply(pd.to_numeric, errors='coerce')

# 3. Fill NaN with median for each column
for col in numerical_cols_present:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)

# 4. Scale the numeric columns
scaler = StandardScaler()
X_train[numerical_cols_present] = scaler.fit_transform(X_train[numerical_cols_present])
X_test[numerical_cols_present] = scaler.transform(X_test[numerical_cols_present])

In [29]:
from sklearn.metrics import r2_score, mean_squared_error
print("\n--- Model Training: Random Forest Regressor ---")
# Initialize and train the Random Forest Regressor model
# Using parameters that generally give good results, but further tuning can be done.
rf_model = RandomForestRegressor(n_estimators=450,n_jobs=-1,random_state=42)
rf_model.fit(X_train, y_train)
print("Random Forest Regressor model trained.")

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print(f"\n--- Model Evaluation (Random Forest) ---")
print(f"R-squared (R^2) Score: {r2_rf:.4f}")
print(f"Mean Squared Error (MSE): {mse_rf:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.4f}")


--- Model Training: Random Forest Regressor ---
Random Forest Regressor model trained.

--- Model Evaluation (Random Forest) ---
R-squared (R^2) Score: 0.9938
Mean Squared Error (MSE): 0.0054
Root Mean Squared Error (RMSE): 0.0737


In [30]:
import pandas as pd
import numpy as np

# Get feature importances
importances = rf_model.feature_importances_

# If X_train is a DataFrame
feature_names = X_train.columns




# Convert to percentage
importances_percentage = 100 * importances / importances.sum()

# Create DataFrame of feature importances
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance (%)': importances_percentage
}).sort_values(by='Importance (%)', ascending=False)

print(feature_importance_df)


                                               Feature  Importance (%)
0                                         size_in_sqft       61.800319
89                              property_type_showroom        8.219595
1                                     carpet_area_sqft        5.522786
5                                         property_age        3.929394
77                          location_hub_business park        2.607076
..                                                 ...             ...
142   amenities_count_parking, CCTV, water storage (3)        0.000000
145  amenities_count_parking, staircase,water stora...        0.000000
161                 amenities_count_security, vastu(2)        0.000000
147    amenities_count_parking, vastu, power backup(3)        0.000000
151          amenities_count_parking, water storage(2)        0.000000

[202 rows x 2 columns]


In [32]:
# --- Saving the trained model and scaler ---
import joblib
print("\n--- Saving Model and Scaler ---")
try:
    joblib.dump(rf_model, 'mc.pkl')
    joblib.dump(scaler, 'sc.pkl')
    joblib.dump(X.columns.tolist(), 'fc.pkl') # Save feature names for consistency
    print("Model, scaler, and feature names saved successfully.")
except Exception as e:
    print(f"Error saving model/scaler: {e}")


--- Saving Model and Scaler ---
Model, scaler, and feature names saved successfully.
